In [157]:
#importation des bibliothèques
import pandas as pd 
import json
import re

In [158]:
#chargement des fichiers CSV
with open('lampadaires.csv', 'r') as f_lamp:
    lamp = pd.read_csv(f_lamp)

with open('led.csv', 'r') as f_led:
    led = pd.read_csv(f_led)


In [159]:
# Mise en forme de la date au format ISO 8601
def to_iso8601(date_str):

    #Si la case du CSV est vide
    if not date_str or pd.isna(date_str):
        return None

    parsed = pd.to_datetime(date_str, errors='coerce')

    #Si la date est déjà au format: "2012-08-25 10:49"
    if pd.notna(parsed):
        return parsed.strftime('%Y-%m-%dT%H:%M')

    #Si la date est au format: "dimanche 03 juillet 2016 à 23:12"
    parts = date_str.split(' ', 1)
    if len(parts) >1:
        date_str = parts[1]
    date_str = date_str.replace(' à ',' ')
    
    mapping = {
        'janvier':'January',
        'février':'February',
        'mars':'March',
        'avril':'April',
        'mai':'May',
        'juin':'June',
        'juillet':'July',
        'août':'August',
        'septembre':'September',
        'octobre': 'October',
        'novembre':'November',
        'décembre':'December'
    }

    for fr, en in mapping.items():
        date_str = date_str.replace(fr, en)
    
    cleaned_date = pd.to_datetime(date_str, format='%d %B %Y %H:%M', errors='coerce')
    if pd.notna(cleaned_date):
        return cleaned_date.strftime('%Y-%m-%dT%H:%M')
    
    return None


In [160]:
# Mise en forme de la couleur des ampoules 
def to_valid_color(color):
    if pd.notna(color):
        color = str(color).lower()
    valid_colors = ["bleu", "blanc", "jaune"]
    if color in valid_colors:
        return color
    return None

In [161]:
# création structure json
json_data = {
    "data": []
}

In [162]:
#ajout des données dans la structure 
for index, l in lamp.iterrows():
    lampadaire = {
        "type": "lampadaire",
        "id": l["Identifiant du support"],  
        "attributes": {},
        "relationships": {
            "leds": {
                "data": []
            }
        }
    }

    # Ajout des attributs à lampadaire seulement si la valeur est valide
    if pd.notna(l["Modèle"]):
        lampadaire["attributes"]["modèle"] = l["Modèle"]
    
    if pd.notna(l['Adresse : Num de la voie']) and pd.notna(l['Nom de la voie']) and pd.notna(l['Ville']):
        lampadaire["attributes"]["addresse"] = f"{l['Adresse : Num de la voie']} {l['Nom de la voie']}, {l['Ville']}"
    
    if pd.notna(l['Maintenance cache']):
        lampadaire["attributes"]["date dernière maintenance"] = to_iso8601(l['Maintenance cache'])
    
    if pd.notna(l['Hauteur du support']):
        lampadaire["attributes"]["hauteurSupport"] = l['Hauteur du support']
    
    if pd.notna(l['Présence appareillage électronique']):
        lampadaire["attributes"]["présenceAppareillage"] = l['Présence appareillage électronique']
    
    if pd.notna(l['Présence console']):
        lampadaire["attributes"]["présenceConsole"] = l['Présence console']

    # Traitement des LEDs
    for i in range(1, 13):
        ampoule = f"Ampoule{i}"
        couleur = f"Couleur de la LED ampoule{i}"
        
        if ampoule in l and pd.notna(l[ampoule]):
            reference = l[ampoule]
            ligne = led[led['Reference'] == reference]
            
            led_data = {
                "type": "led",
                "id": int(l[ampoule]),
                "attributes": {}
            }

            if pd.notna(l.get(couleur)):
                led_data["attributes"]["couleur"] = to_valid_color(l[couleur])
            
            if pd.notna(ligne['Puissance consommée'].iloc[0]):
                led_data["attributes"]["puissance"] = ligne['Puissance consommée'].iloc[0].item()
            
            if pd.notna(ligne['Maintenance LED'].iloc[0]):
                led_data["attributes"]["date maintenance"] = to_iso8601(ligne['Maintenance LED'].iloc[0])
            
            if pd.notna(ligne['Libellé'].iloc[0]):
                led_data["attributes"]["libellé"] = ligne['Libellé'].iloc[0]
            
            if pd.notna(ligne['Durée de vie'].iloc[0]):
                led_data["attributes"]["duréeDeVie"] = ligne['Durée de vie'].iloc[0].item()
            
            if pd.notna(ligne['Unité de temps pour la durée de vie'].iloc[0]):
                led_data["attributes"]["unitéDeTemps"] = ligne['Unité de temps pour la durée de vie'].iloc[0].item()

            lampadaire['relationships']['leds']['data'].append(led_data)

    json_data["data"].append(lampadaire)

In [163]:
#création du fichier json
with open("result.json", "w", encoding="utf-8") as res:
    res.write(json.dumps(json_data, indent=2, ensure_ascii=False))